In [1]:
%matplotlib widget

In [2]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import ipyleaflet as leaf
import ipywidgets as ipyw
from shapely.geometry import Polygon
import sqlite3 as sql
from matplotlib.dates import YearLocator


In [3]:
#gets the data for the world map from geopandas
europe = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))

#remove countries that arent part of europe
europe=europe[europe.continent == "Europe"]
europe=europe[(europe.name != "Russia")]

#creates a bounding box to remove parts of countries that arent in europe, e.g. french guiana
polygon = Polygon([(-25,35), (40,35), (40,75),(-25,75)])
poly_gdf = gpd.GeoDataFrame([1], geometry=[polygon], crs=europe.crs)
europe = gpd.clip(europe, polygon)

/tmp/ipykernel_12729/350249023.py:2: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  europe = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))


In [12]:
con = sql.connect('countries2')
#defines function for querying the database
def get_country_data(country_name='Slovakia'):
    query1a = "SELECT Date, Petrol, Diesel FROM Countries WHERE name = ?"
    try:
        data = pd.read_sql_query(query1a, con, params=(country_name,))
        return data
    except pd.errors.EmptyDataError:
        print(f"No data found for {country_name}.")
        return None

#creates a base state for the graph that is not interactive
with plt.ioff():
    fig = plt.figure(figsize=(6,4))
    ax = fig.add_subplot(1, 1, 1)

    country = 'Germany'
    
    country_df = get_country_data(country)
    # Check if data is available before proceeding
    if country_df is not None:
    
        country_df['Date'] = pd.to_datetime(country_df['Date'], dayfirst=True)
        # Remove commas from the 'Petrol' and 'Diesel' columns
        country_df['Petrol'] = country_df['Petrol'].replace({',': ''}, regex=True)
        country_df['Diesel'] = country_df['Diesel'].str.replace(',', '')

        country_df['Petrol'] = pd.to_numeric(country_df['Petrol'])
        country_df['Diesel'] = pd.to_numeric(country_df['Diesel'])

        df_subset = country_df.iloc[0:]
        # Create a plot
        #plt.figure(figsize=(10, 6))
        ax.plot(df_subset['Date'], df_subset['Petrol'], label='Petrol', color='red')
        ax.plot(df_subset['Date'], df_subset['Diesel'], label='Diesel', color='blue')

        # Adjust x-axis ticks
        plt.xticks(df_subset['Date'][::50], rotation=45)

        plt.gca().xaxis.set_major_locator(YearLocator())
        # Invert x-axis
    
        plt.gca().invert_xaxis()

        # Add labels, limits and title
        start_date = pd.to_datetime('2004-12-30')
        end_date = pd.to_datetime('2023-12-01')
        plt.xlim(start_date, end_date)
        plt.ylim(0, 1800)
        plt.xlabel('Year', size = 12)
        plt.ylabel('Price (€)', size = 12)
        plt.title(f'{country}', size = 18)

        # Adjust legend placement
        plt.legend(loc='upper left', fontsize = 11)
    #formatting of graph
    fig.canvas.toolbar_visible=False
    fig.canvas.header_visible=False
    fig.canvas.footer_visible=False
    fig.canvas.resizable=False

    plt.tight_layout()

#a function to update the graph when a country is clicked
def update_graph(country):
    con = sql.connect('countries2')
    ax.clear()
    country_df = get_country_data(country)
    # Check if data is available before proceeding
    if country_df is not None:
    
        country_df['Date'] = pd.to_datetime(country_df['Date'], dayfirst=True)
        # Remove commas from the 'Petrol' and 'Diesel' columns
        country_df['Petrol'] = country_df['Petrol'].replace({',': ''}, regex=True)
        country_df['Diesel'] = country_df['Diesel'].str.replace(',', '')

        country_df['Petrol'] = pd.to_numeric(country_df['Petrol'])
        country_df['Diesel'] = pd.to_numeric(country_df['Diesel'])

        df_subset = country_df.iloc[0:]
        # Create a plot
        ax.plot(df_subset['Date'], df_subset['Petrol'], label='Petrol', color = 'red')
        ax.plot(df_subset['Date'], df_subset['Diesel'], label='Diesel', color = 'blue')

        # Adjust x-axis ticks
        plt.xticks(df_subset['Date'][::50], rotation=45)

        plt.gca().xaxis.set_major_locator(YearLocator())
        # Invert x-axis
    
        plt.gca().invert_xaxis()

        # Add labels, limits and title
        start_date = pd.to_datetime('2004-12-30')
        end_date = pd.to_datetime('2023-12-01')
        plt.xlim(start_date, end_date)
        plt.ylim(0, 1800)
        plt.xlabel('Year', size = 12)
        plt.ylabel('Price (€)', size = 12)
        plt.title(f'{country}', size = 18)

        # Adjust legend placement
        plt.legend(loc='upper left', fontsize = 11)

    con.close()

#defines the ipyleaflet basemap
m = leaf.Map(center = (56, 10), 
             zoom = 3, 
             layout=ipyw.Layout(width='45%', height='350px'),
             min_zoom = 3,
             scroll_wheel_zoom = True,
             zoom_control = False
             )

#uses the data from the europe map to create an overlay for the interactivity
geo_data = leaf.GeoData(geo_dataframe = europe,
                   style={'fillColor':'green','color':'black','opacity':0.6, 'weight':2, 'dashArray':'0', 'fillOpacity':0.6},
                   hover_style={'fillColor': 'green' ,'color':'black', 'fillOpacity': 0.2},
                   name = 'Countries')

m.add_layer(geo_data)

#defines what should happen when a country is clicked
def when_clicked(event, feature, **kwargs):
    update_graph(feature['properties']['name'])

#calls the function to be used on click
geo_data.on_click(when_clicked)

#draws the layout of the display
ipyw.HBox([m ,fig.canvas], layout=ipyw.Layout(flex='1 1', width='70%'))


/tmp/ipykernel_12729/4129643154.py:23: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  country_df['Date'] = pd.to_datetime(country_df['Date'], dayfirst=True)


/tmp/ipykernel_12729/4129643154.py:72: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  country_df['Date'] = pd.to_datetime(country_df['Date'], dayfirst=True)
/tmp/ipykernel_12729/4129643154.py:72: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  country_df['Date'] = pd.to_datetime(country_df['Date'], dayfirst=True)
/tmp/ipykernel_12729/4129643154.py:72: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  country_df['Date'] = pd.to_datetime(country_df['Date'], dayfirst=True)
/tmp/ipykernel_12729/4129643154.py:72: UserWarning: Could not infer format, so each element will be parsed indiv